In [1]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
torch.cuda.is_available()

True

In [2]:
torch.cuda.device_count()

1

In [3]:
GPU_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Processing Data

dataset credits: https://www.kaggle.com/datasets/mostafaabla/garbage-classification


In [4]:
import splitfolders

In [5]:
trashData = "TrashData"

# splitfolders.ratio("garbage_classification", trashData, seed=42, ratio = (.8, .2), group_prefix = None)


In [6]:
# is there a way I can create an input pipeline for the data?
# will augment data, etc

In [7]:
trashTrainDataPath = "./TrashData/train"
trashTestDataPath = "./TrashData/val"

In [8]:
import torchvision.transforms as transforms

In [9]:
SIZE = (128,128)
transform = transforms.Compose(
    [transforms.Resize(SIZE),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

In [10]:
train_dataset = torchvision.datasets.ImageFolder(root = trashTrainDataPath, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [11]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [12]:
test_dataset = torchvision.datasets.ImageFolder(root = trashTestDataPath, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Training the Model

In [13]:
from TrashModel import TrashModel

In [14]:
trashModel = TrashModel(GPU_DEVICE)
trashModel.cuda()
trashModel.train_custom_model(train_loader=train_loader)    

/home/shawn/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shawn/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [27]:
torch.save(trashModel.state_dict(), './TrainingAccuracyWeights/model1_weights')

# Verifying Model

In [36]:
import os

# Path to your test directory
test_dir = './TrashData/val/'

# Get a list of subdirectories (labels)
labels = [label for label in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, label))]

labels = labels[::-1]
labels


['biological',
 'battery',
 'paper',
 'clothes',
 'trash',
 'cardboard',
 'shoes',
 'plastic',
 'green-glass',
 'metal',
 'white-glass',
 'brown-glass']

In [ ]:


# Load the trained model
model = TrashModel()
model.load_state_dict(torch.load('./TrainingAccuracyWeights/model1_weights'))
model.eval()

# Prepare a test data loader
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=..., shuffle=False)

# Define a function to get class labels from class indices
def get_class_labels(class_indices,class_labels):
    return [class_labels[idx] for idx in class_indices]

# Iterate through test loader batches
for inputs, labels in test_loader:
    with torch.no_grad():
        outputs = model(inputs)
    
    _, predicted_classes = torch.max(outputs, 1)
    predicted_labels = get_class_labels(predicted_classes.tolist(), labels)
    
    # Print or process predicted labels
    print(predicted_labels)
